# Simulator runner

In [1]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [2]:
from utilitysimv2 import InferenceJob, TrainingJob, get_optimus_fn, get_tanh_fn, get_linear_fn, get_tanh_fn, k_optimus_fn, simulator, thief_scheduler, single_camera_optimizer, thief_sco_scheduler, thief_scheduler

In [3]:
def generate_config(name, conv_time, scale, target_start_accuracy = 50, inference_job = None):
    func, inv_func = get_optimus_fn(conv_time, scale)
    init_time = inv_func(target_start_accuracy)
    #print(init_time)
    if init_time > conv_time:
        raise Exception("The model is already trained. init_time {} > conv_time {}".format(init_time, conv_time))
    return TrainingJob(name, func, init_time, conv_time, resource_alloc=0, inference_job=inference_job)
  

def generate_config2(name, final_accuracy, job_time, start_accuracy = 50, inference_job = None):
    if start_accuracy > final_accuracy:
        raise Exception("The model is already trained. start_accuracy {} > final_accuracy {}".format(start_accuracy, final_accuracy))
    # solve a=1 / ((1 / (t * 10 * 1 / (t+d))) + 1) * 100 * K, b=1 / ((1 / ((t+d) * 10 * 1 / (t+d))) + 1) * 100 * K for K, t
    a=start_accuracy
    b=final_accuracy
    d=job_time
    k=11*b/1000
    t=a*d/(1000*k - 11*a)
    conv_time=t+d
    
    func, inv_func = get_optimus_fn(conv_time, k)
    init_time = inv_func(start_accuracy)
    return TrainingJob(name, func, init_time, conv_time, resource_alloc=0, inference_job=inference_job)

## Single job for debugging

In [4]:
target_start_accuracy = 20
total_res = 1
retraining_period = 200

A_inference = InferenceJob("A_inference", target_start_accuracy, get_linear_fn(1)[0], resource_alloc=0)

A_training_configs = [generate_config2("train_1", 90, 300, target_start_accuracy, inference_job=A_inference),
                   generate_config2("train_2", 80, 100, target_start_accuracy, inference_job=A_inference),
                   #generate_config2("train_3", 70, 50, target_start_accuracy, inference_job=A_inference),
                   #generate_config2("train_4", 60, 45, target_start_accuracy, inference_job=A_inference),
                   generate_config2("train_5", 65, 150, target_start_accuracy, inference_job=A_inference)]

In [5]:
best_trg_config, debug = single_camera_optimizer(A_training_configs, A_inference, total_resources=total_res,
                                                 retraining_period=retraining_period,
                                                 training_res_alloc = 0.5,
                                                 inference_res_alloc = 0.5)
print(best_trg_config)
print(debug)
print("Warning - this assumes fixed resources 0.5 and 0.5")

train_2
[{0: 10.0, 1: 10.34843352782209, 2: 10.0}]
Warning - this assumes fixed resources 0.5 and 0.5


In [6]:
a_func, a_inv_func = get_optimus_fn(100)
init_time_a = a_inv_func(1)
A_train = TrainingJob("A_Train", a_func, init_time_a, 100, resource_alloc=0, inference_job=None)

In [7]:
job_pairs = [(A_inference, A_training_configs)]
period_allocation = thief_sco_scheduler(job_pairs, total_res, retraining_period, iterations = 3)

Iter 0, alloc: {<utilitysimv2.InferenceJob object at 0x7f300e497080>: [0.5, 0.5]}
Best config for A_inference: train_2
Iter 0, final_alloc: {<utilitysimv2.InferenceJob object at 0x7f300e497080>: [0.0, 1.0]}
Iter 1, alloc: {<utilitysimv2.InferenceJob object at 0x7f300e497080>: [0.0, 1.0]}
Best config for A_inference: train_2
Iter 1, final_alloc: {<utilitysimv2.InferenceJob object at 0x7f300e497080>: [0.0, 1.0]}
Iter 2, alloc: {<utilitysimv2.InferenceJob object at 0x7f300e497080>: [0.0, 1.0]}
Best config for A_inference: train_2
Iter 2, final_alloc: {<utilitysimv2.InferenceJob object at 0x7f300e497080>: [0.0, 1.0]}


In [8]:
period_allocation

{0: {'train_2': 1.0, 'A_inference': 0.0},
 100.0: {'train_2': 0, 'A_inference': 1.0}}

In [16]:
training_jobs = [next(a for a in A_training_configs if a.name == 'train_2')]
inference_jobs = [A_inference]

In [17]:
sim = simulator(training_jobs, inference_jobs, 1, retraining_period=retraining_period, period_allocation=period_allocation)
results = sim.step_till_completion()
print(results['meta'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
Simulator steps done.


{'inf_mean_auc': 45.620183139975076}


In [24]:
period_allocation = {0: {'train_2': 1, 'A_inference': 0},
                    100: {'train_2': 0, 'A_inference': 1}}

training_jobs = [next(a for a in A_training_configs if a.name == 'train_2')]
inference_jobs = [A_inference]
sim = simulator(training_jobs, inference_jobs, 1, retraining_period=retraining_period, period_allocation=period_allocation)
results = sim.step_till_completion()
print(results['meta'])

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Job train_2 is done.
100
Job train_2 is done.
101
Job train_2 is done.
102
Job train_2 is done.
103
Job train_2 is done.
104
Job train_2 is done.
105
Job train_2 is done.
106
Job train_2 is done.
107
Job train_2 is done.
108
Job train_2 is done.
109
Job train_2 is done.
110
Job train_2 is done.
111
Job train_2 is done.
112
Job train_2 is done.
113
Job train_2 is done.
114
Job train_2 is done.
115
Job train_2 is done.
116
Job train_2 is done.
117
Job train_2 is done.
118
Job train_2 is done.
119
Job train_2 is done.
120
Job train_2 is done.
121
Job train_2 is done.
122
Job train_2 is done.
123
Job train_2 is done.
124
Job train_2 is done.
125
Job train_2 is done.
126
Job train_2 is done.
127
Job train_2 

{'inf_mean_auc': 40.234164454694486}


## Two job test

In [4]:
total_res = 1
retraining_period = 200
start_accuracy_a = 20
start_accuracy_b = 10

A_inference = InferenceJob("A_inference", start_accuracy_a, get_linear_fn(1)[0], resource_alloc=0)
A_training_configs = [generate_config2("A_train_1", 90, 300, start_accuracy_a, inference_job=A_inference),
                   generate_config2("A_train_2", 80, 100, start_accuracy_a, inference_job=A_inference),
                   generate_config2("A_train_3", 30, 50, start_accuracy_a, inference_job=A_inference),
                   generate_config2("A_train_4", 25, 31, start_accuracy_a, inference_job=A_inference)]

B_inference = InferenceJob("B_inference", start_accuracy_b, get_linear_fn(1)[0], resource_alloc=0)
B_training_configs = [generate_config2("B_train_1", 90, 300, start_accuracy_b, inference_job=B_inference),
                   generate_config2("B_train_2", 70, 100, start_accuracy_b, inference_job=B_inference),
                   generate_config2("B_train_3", 30, 50, start_accuracy_b, inference_job=B_inference),
                   generate_config2("B_train_4", 80, 30, start_accuracy_b, inference_job=B_inference)]

job_pairs = [(A_inference, A_training_configs), (B_inference, B_training_configs)]

In [5]:
period_allocation = thief_sco_scheduler(job_pairs, total_res, retraining_period, iterations = 5)

Iter 0, alloc: {<utilitysimv2.InferenceJob object at 0x7f2d0e9e00f0>: [0.25, 0.25], <utilitysimv2.InferenceJob object at 0x7f2d0e9e0128>: [0.25, 0.25]}
Best config for A_inference: A_train_2
Best config for B_inference: B_train_4
Iter 0, final_alloc: {<utilitysimv2.InferenceJob object at 0x7f2d0e9e00f0>: [0.04999999999999999, 0.04999999999999999], <utilitysimv2.InferenceJob object at 0x7f2d0e9e0128>: [0.65, 0.25]}
Iter 1, alloc: {<utilitysimv2.InferenceJob object at 0x7f2d0e9e00f0>: [0.04999999999999999, 0.04999999999999999], <utilitysimv2.InferenceJob object at 0x7f2d0e9e0128>: [0.65, 0.25]}
Best config for A_inference: A_train_2
Best config for B_inference: B_train_4
Iter 1, final_alloc: {<utilitysimv2.InferenceJob object at 0x7f2d0e9e00f0>: [0.04999999999999999, 0.04999999999999999], <utilitysimv2.InferenceJob object at 0x7f2d0e9e0128>: [0.65, 0.25]}
Iter 2, alloc: {<utilitysimv2.InferenceJob object at 0x7f2d0e9e00f0>: [0.04999999999999999, 0.04999999999999999], <utilitysimv2.Infere

In [6]:
period_allocation

{0: {'A_train_2': 0.04999999999999999,
  'B_train_4': 0.25,
  'A_inference': 0.04999999999999999,
  'B_inference': 0.65},
 120.0: {'B_train_4': 0,
  'A_train_2': 0.1333333333333333,
  'A_inference': 0.1333333333333333,
  'B_inference': 0.7333333333333334},
 825.0000000000001: {'A_train_2': 0,
  'B_train_4': 0,
  'A_inference': 0.19999999999999996,
  'B_inference': 0.8}}

# Visualization

### Accuracy progression

In [ ]:
plt.figure()
for job, result in results.items():
    if job != 'meta':
        t, acc = list(zip(*result))
        plt.plot(t, acc, label=job.name)
plt.title("Training and inference job accuracy over time")
plt.legend()
plt.xlabel("Wall time")
plt.ylabel("Accuracy")

In [ ]:
for t, allocation in sim.period_allocation.items():
    names = []
    allocs = []
    for j, alloc in allocation.items():
        allocs.append(alloc)
        names.append(j.name)
    print(names, allocs)
    plt.figure()
    plt.pie(allocs, labels=names, autopct='%1.1f%%',
        shadow=True, startangle=90)
    plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle
    plt.title("Allocation at t = {}".format(t))

In [ ]:
means = []
for j, result in results.items():
    if isinstance(j, InferenceJob):
        mean_accuracy = sum(r[1] for r in result)/len(result)
        means.append(mean_accuracy)
MEAN_AUC = sum(means)/len(means)
print("Mean Inference Accuracy across all jobs: {}".format(MEAN_AUC))

In [ ]:
results['meta']